In [ ]:
!pip install pulp
!pip install cplex
import pandas as pd
import numpy as np
import pulp as plp
import time
import warnings
warnings.filterwarnings('ignore')

     |████████████████████████████████| 14.2 MB 21.7 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
     |████████████████████████████████| 43.3 MB 4.2 MB/s 
You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
def GetWeighted():
    """Funcion para obtener los pesos de las habilidades por posicion
    Returns:
        Weighted_Skills(pandas Data frame): Dataframe con el peso de las habilidades por posición
    """
    #Se obtiene la información de la página web mostrada y se limpian los datos
    table_MN = pd.read_html('https://www.bradleymonk.com/wp/ea-fifa-21-player-ratings/')
    Weighted_Skills=table_MN[0]
    Weighted_Skills[0][0]='Skill'
    Weighted_Skills.columns = Weighted_Skills.iloc[0]
    Weighted_Skills = Weighted_Skills.iloc[1:].reset_index(drop=True)
    Weighted_Skills.replace('–', 0.0, inplace=True)
    Weighted_Skills.columns = Weighted_Skills.columns.fillna('temp')
    Weighted_Skills.drop('temp', axis = 1, inplace = True)
    Weighted_Skills= Weighted_Skills.dropna()

    #Se trabajan los datos para que sumen 1, esto porque es necesario para el modelo
    Weighted_Skills.at[17, 'ST'] = float(Weighted_Skills.at[17, 'ST']) - 0.02 
    Weighted_Skills.at[17, 'CM'] = float(Weighted_Skills.at[17, 'CM']) - 0.01 
    Weighted_Skills.at[17, 'CDM'] = float(Weighted_Skills.at[17, 'CDM']) - 0.02 
    Weighted_Skills.at[17, 'L/RM'] = float(Weighted_Skills.at[17, 'L/RM']) - 0.01 
    Weighted_Skills.at[17, 'L/RB'] = float(Weighted_Skills.at[17, 'L/RB']) - 0.02 
    Weighted_Skills.at[17, 'L/RW'] = float(Weighted_Skills.at[17, 'L/RW']) - 0.02
    Weighted_Skills.at[17, 'CB'] = float(Weighted_Skills.at[17, 'CB']) - 0.02  
    Weighted_Skills.at[21, 'L/RB'] = float(Weighted_Skills.at[21, 'L/RB']) - 0.01

    #Se añaden las posiciones derechas e izquierdas, a partir de las columnas L/R
    Weighted_Skills['LM'] = Weighted_Skills['L/RM'] 
    Weighted_Skills.rename(columns={'L/RM': 'RM'}, inplace=True)
    Weighted_Skills['LW'] = Weighted_Skills['L/RW'] 
    Weighted_Skills.rename(columns={'L/RW': 'RW'}, inplace=True)
    Weighted_Skills['LB'] = Weighted_Skills['L/RB'] 
    Weighted_Skills.rename(columns={'L/RB': 'RB'}, inplace=True)

    #Se añade la posición de portero y sus pesos
    goalkeeper=[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0, 0, 0.11,0,0,0,0,0,0,0,0,0,0,0,0,0]
    gkdf= pd.DataFrame([['gk_diving',0,0,0,0,0,0,0,0,0,0,0, 0.21], ['gk_handling',0,0,0,0,0,0,0,0,0,0,0, 0.21],\
                        ['gk_kicking',0,0,0,0,0,0,0,0,0,0,0, 0.05], ['gk_positioning',0,0,0,0,0,0,0,0,0,0,0, 0.21],\
                        ['gk_reflexes',0,0,0,0,0,0,0,0,0,0,0, 0.21]])
    Weighted_Skills['GK']= goalkeeper
    gkdf.columns= Weighted_Skills.columns
    Weighted_Skills=pd.concat([gkdf, Weighted_Skills], axis=0)
    Weighted_Skills.drop(Weighted_Skills.tail(1).index, inplace=True)
    Weighted_Skills.reset_index(inplace=True, drop=True)

    return(Weighted_Skills)


def GetPlayers(Weighted_Skills, price, better):
    """Funcion para obtener los jugadores de los dataframes de la FIFA
    Args:
        Weighted_Skills(pandas Dataframe): el dataframe de las skills que se van a comparar ya que el orden de las columnas de ambos debe ser el mismo
        price(float): El precio máximo por jugador.

    Returns:
        dataframe2(pandas Dataframe): El dataframe de los jugadors con las estadisticas usadas por nuestro modelo
    """
    #Se lee el csv solicitado se llenan los precios 0 tomando la media de los precios y multiplicando por el overall del jugador
    #Los precios 0 son debido a que durante el lanzamiento de los datos el contrato estaba terminado por lo que no se tenia un número fijo
    year=20
    file_string= 'players_'+str(year)+'.csv'
    dataframe = pd.read_csv(file_string)
    dataframe['value_eur'] = np.where(dataframe['value_eur'] == 0, (dataframe['overall']*dataframe['value_eur'].mean())/100, dataframe['value_eur'])
    
    #Se filtra por precio y como buscamos el mejor equipo sobre el precio obtenemos a los jugadores que estén por arriba del 70%
    dataframe=dataframe[dataframe['value_eur']<=price]
    dataframe['overall']=dataframe['overall']/dataframe['overall'].max()
    dataframe= dataframe[dataframe['overall']>better]
    dataframe['player_positions']=dataframe['player_positions'].str.split(',').str[0]

    #Dejamos los atributos estrictamente necesarios
    atributos = ['long_name',
    'value_eur',
    'player_positions',
    'gk_diving',
    'gk_handling',
    'gk_kicking',
    'gk_reflexes',
    'gk_positioning',
    'attacking_crossing',
    'attacking_finishing',
    'attacking_heading_accuracy',
    'attacking_short_passing',
    'attacking_volleys',
    'skill_dribbling',
    'skill_curve',
    'skill_fk_accuracy',
    'skill_long_passing',
    'skill_ball_control',
    'movement_acceleration',
    'movement_sprint_speed',
    'movement_agility',
    'movement_reactions',
    'movement_balance',
    'power_shot_power',
    'power_jumping',
    'power_stamina',
    'power_long_shots',
    'mentality_aggression',
    'mentality_interceptions',
    'mentality_positioning',
    'mentality_vision',
    'mentality_penalties',
    'mentality_composure',
    'defending_marking',
    'defending_standing_tackle',
    'defending_sliding_tackle',
    ]

    #Le cambiamos los nombres a los atributos a los que tenemos en el dataframe de skills
    dataframe=dataframe[atributos]
    dataframe.rename(columns={'movement_acceleration':'Acceleration', 'movement_sprint_speed':'SprintSpeed',\
        'mentality_positioning': 'Positioning', 'attacking_finishing':'Finishing', 'power_shot_power':'ShotPower',\
        'power_long_shots':'LongShots', 'attacking_volleys':'Volleys', 'mentality_penalties':'Penalties',\
        'mentality_vision': 'Vision', 'attacking_crossing':'Crossing', 'skill_fk_accuracy':'FKAccuracy',\
        'attacking_short_passing':'ShortPassing', 'skill_long_passing':'LongPassing',\
        'skill_curve':'Curve', 'movement_agility':'Agility', 'movement_balance':'Balance',\
        'movement_reactions':'Reactions', 'skill_ball_control':'BallControl', 'skill_dribbling':'Dribbling',\
        'mentality_composure':'Composure', 'mentality_interceptions':'Interceptions',\
        'attacking_heading_accuracy':'HeadAccuracy', 'defending_marking':'DefAwareness',\
        'defending_standing_tackle':'StandingTackle','defending_sliding_tackle':'SlidingTackle',\
        'power_jumping':'Jumping', 'power_stamina':'Stamina', 'mentality_aggression':'Aggression',
        }, inplace=True)

    #Cambiamos el nombre de las columnas y llenamos los ceros
    dataframe2=dataframe.drop(['long_name', 'player_positions','value_eur'], axis=1)
    dataframe2=dataframe2.reindex(columns=Weighted_Skills['Skill'].tolist())
    dataframe2['Name']=dataframe['long_name']
    dataframe2['Position']=dataframe['player_positions']
    dataframe2['Cost']= dataframe['value_eur']
    dataframe2.reset_index(inplace=True, drop=True)
    dataframe2=dataframe2.fillna(0)

    return(dataframe2)



def GetPositions(dataframe2, Weighted_Skills):
    """Función que obtiene un dataframe indicando las posiciones posibles por jugador
    Args:
        dataframe2(pandas Dataframe): El dataframe de donde se van a sacar los jugadores y las posiciones que juegan
        WeightedSkills (pandas Dataframe): El dataframe de donde se van a sacar las posiciones que consideramos
    Returns:
        df_positions(pandas Dataframe): Un dataframe con las posiciones posibles por jugador
    """
    #Creamos nuestro dataframe insertando una columna de nombres
    df_positions= pd.DataFrame(columns= np.insert(Weighted_Skills.columns[1:], 0, 'Name'))
    index=0
    #Iteramos sobre el numero de jugadores
    for player in dataframe2['Name']:
        positions=[]
        positions.append(player)
        #Iteramos sobre el numero de posiciones
        for position in Weighted_Skills.columns[1:]:
            #Obtenemos un valor de verdad sobre la posibilidad de un jugador de jugar en una posición dada
            positions.append(position==dataframe2.at[index, 'Position'])
        index+=1
        #Añadimos al datarame dicho valor
        df_positions.loc[index] = positions
    return(df_positions)


def GetWPos():
    """Función que obtiene un dataframe con el peso de las alineaciones.
    Returns:
        Weighted_positions(pandas Dataframe): Dataframe con el peso de cada posición
    """
    #Debido a la escasez de información de los pesos sobre las formaciones, de manera arbitraria
    #Le dimos un peso a una formación; la idea es que sea posible tener distintas formaciones
    #Para esto creamos el dataframe de manera manual

    weighted_position={'ST':[0.15],'CAM':[0.10], 	'CM':[0.10], 	'RW':[0.10], 	'RM':[0.5] 	,\
                    'RB':[0.5], 	'CB':[0.10], 	'LM':[0.5], 	'LW':[0.10], 	'LB':[0.5], 	'GK':[0.15]}
                    
    Weighted_positions=pd.DataFrame.from_dict(weighted_position)
    return(Weighted_positions)


def GetTeam(price=10000000000, exception=[], results=1, better=80):
    """Funcion que obtiene el mejor equipo posible dada ciertas restricciones
    Args:
        year(int): Año del dataframe donde se van a obtener los jugadores y sus datos
        price(float): Precio máximo de cada jugador
        exception(array): Una arreglo de strings de donde se descartaran jugadores que no queremos en nuestro equipo
        results(int): cantidad de equipos distintos que queremos obtener
    
    Returns:
        Solution(array): Un arreglo de arreglos que contiene los equipos formados y el precio y la posicion de cada jugador
    """

    #Primero obtenemos los dataframes necesarios, los pesos y las posiciones
    wsdf= GetWeighted()
    playdf=GetPlayers(wsdf, price, better)
    #Eliminamos a los jugadores que no queremos
    for ex in exception:
        playdf=playdf[playdf.Name != ex]
    playdf.reset_index(inplace=True, drop=True)
    wpdf=GetWPos()
    posdf=GetPositions(playdf, wsdf)

    #Inicializamos los parametros para nuestro modelo lineal
    n=len(wpdf.columns)
    k=len(wsdf)
    set_j= range(0, n)
    set_y= range(0, k)
    solution=[]

    #Iteramos sobre la cantidad de resultados solicitados
    for s in range(0, results):
        print('Buscando el mejor equipo de entre {0} jugadores'.format(len(playdf)))
        #Pasamos los datos a un diccionario para poder usarlos en nuestro modelo
        m=len(playdf)
        set_i= range(0, m)
        SSiy={(i,y): float(playdf.iat[i,y]) for i in set_i for y in set_y}
        SMyj= {(y,j): float(wsdf.iat[y,j+1]) for y in set_y for j in set_j}
        MSj= { j: float(wpdf.iat[0,j]) for j in set_j}
        PMij={(i,j): float(posdf.iat[i, j+1]) for i in set_i for j in set_j}
        pairs=[(i,j) for i in set_i for j in set_j]

        #Inicializamos nuestro problema a optimizar
        opt_mod = plp.LpProblem("MIP Model", plp.LpMaximize)

        #Inicializamos nuestra variable de decisión
        x_vars = plp.LpVariable.dicts("x", pairs, cat=plp.LpBinary)

        #Damos las restricciones
        for i in set_i:
            opt_mod+=plp.lpSum(x_vars[i,j] for j in set_j) <=1.0
        for j in set_j:
            opt_mod+=plp.lpSum(x_vars[i,j] for i in set_i)==1.0
        for i in set_i:
            for j in set_j:
                opt_mod += x_vars[i,j] <= int(PMij[i,j])
        #Damos nuestra función objetivo
        opt_mod += plp.lpSum(SSiy[i,y]*SMyj[y,j]*MSj[j]* x_vars[i,j] for i in set_i for y in set_y for j in set_j)
        
        #Resolvemos 
        opt_mod.solve()
        
        #Capturamos el resultado y en eliminamos de los datos a los jugadores rpeviamente utilizados para
        #Poder dar un equipo distinto
        result=[]
        remove=[]
        for x in x_vars:
            if(x_vars[x].value()!=0):
                result.append([wpdf.columns[x[1]], playdf.at[x[0], 'Name'], playdf.at[x[0], 'Cost']])
                remove.append(playdf.at[x[0], 'Name'])
        for rem in remove:
            playdf=playdf[playdf.Name != rem]
        playdf.reset_index(inplace=True, drop=True) 
        solution.append(result)
        print("{0} de {1} equipos encontrados".format(s+1, results))

    return(solution)


def DisplaySol(team):
    for k in range(0,len(team)):
        print('---  equipo %s ---' % (k))
        for player in team[k]:
            print(player)







In [ ]:
#Obteniendo el mejor equipo
#Nota: para agilizar el proceso estamos considerando únicamente el top 85% de los jugadores
#Nota, el parametro better funciona decente incluso con el 70% el algoritmo puede buscar equipos optimamente
#con hasta 6000 jugadores, pero las consultas tardan mas de 3 minutos
DisplaySol(GetTeam(better=0.85))

Buscando el mejor equipo de entre 568 jugadores
1 de 1 equipos encontrados
---  equipo 0 ---
['RM', 'Lionel Andrés Messi Cuccittini', 95500000.0]
['ST', 'Cristiano Ronaldo dos Santos Aveiro', 58500000.0]
['LB', 'Neymar da Silva Santos Junior', 105500000.0]
['CAM', 'Kevin De Bruyne', 90000000.0]
['LM', 'Virgil van Dijk', 78000000.0]
['CM', 'Luka Modrić', 45000000.0]
['RW', "N'Golo Kanté", 66000000.0]
['GK', 'Jordi Alba Ramos', 40000000.0]
['CB', 'Daniel Carvajal Ramos', 38000000.0]
['RB', 'Jadon Sancho', 44500000.0]
['LW', 'Douglas Costa de Souza', 31500000.0]


In [ ]:
#Ahoraa obtenemos los resultado sobre el top 85% de los jugadores que ganan 500000 o menos
DisplaySol(GetTeam(500000, better=0.85))

Buscando el mejor equipo de entre 1875 jugadores
1 de 1 equipos encontrados
---  equipo 0 ---
['LM', 'Olivier Deschacht', 475000.0]
['CB', 'Carlos Luciano Araujo', 425000.0]
['RW', 'Fernando Seoane Antelo', 500000.0]
['GK', 'Edson Gérson Torres Guerra', 500000.0]
['RB', 'Chris Burke', 450000.0]
['CM', 'Daniele Croce', 325000.0]
['ST', 'Chris Wondolowski', 375000.0]
['CAM', 'Paulo Bernard Daineiro Cruz', 450000.0]
['RM', 'Ciprian Ioan Deac', 475000.0]
['LW', 'Luis Carlos Arias Cardona', 375000.0]
['LB', '임상협 廉桑', 425000.0]


In [ ]:
#Haciendo la misma consulta que en el caso anterior pero esta vez queremos armar nuestro equipo sin Oliver y Carlos
exception=["Olivier Deschacht", "Carlos Luciano Araujo"]
DisplaySol(GetTeam(500000, exception, better=0.85))

Buscando el mejor equipo de entre 1873 jugadores
1 de 1 equipos encontrados
---  equipo 0 ---
['RW', 'Fernando Seoane Antelo', 500000.0]
['LM', 'Yalçın Ayhan', 350000.0]
['GK', 'Edson Gérson Torres Guerra', 500000.0]
['CB', 'Dagoberto Josué Pinho Parrela', 500000.0]
['RB', 'Chris Burke', 450000.0]
['CM', 'Daniele Croce', 325000.0]
['ST', 'Chris Wondolowski', 375000.0]
['CAM', 'Paulo Bernard Daineiro Cruz', 450000.0]
['RM', 'Ciprian Ioan Deac', 475000.0]
['LW', 'Luis Carlos Arias Cardona', 375000.0]
['LB', '임상협 廉桑', 425000.0]


In [ ]:
DisplaySol(GetTeam(800000, results=3, better=0.85))

Buscando el mejor equipo de entre 2736 jugadores
1 de 3 equipos encontrados
Buscando el mejor equipo de entre 2725 jugadores
2 de 3 equipos encontrados
Buscando el mejor equipo de entre 2714 jugadores
3 de 3 equipos encontrados
---  equipo 0 ---
['LM', 'Pedro Mario Álvarez Abrante', 800000.0]
['CB', 'Marcos Alberto Angeleri', 725000.0]
['CM', '远藤 保仁', 775000.0]
['RW', 'Guillaume Gillet', 750000.0]
['GK', 'Heitor Edvaldo Barreto Gesser', 675000.0]
['ST', 'Umut Bulut', 625000.0]
['CAM', 'Wes Hoolahan', 600000.0]
['LB', '염기훈 廉基勋', 600000.0]
['RB', 'Stewart Downing', 525000.0]
['LW', 'George Boyd', 650000.0]
['RM', 'Pedro Muñoz', 550000.0]
---  equipo 1 ---
['LM', 'Carlos Luciano Araujo', 425000.0]
['RW', 'Farid Alfonso Díaz Rhenals', 675000.0]
['GK', 'Márcio Anderson Tramontino Sá', 675000.0]
['CB', 'João Rodolfo Bardinho Barros', 675000.0]
['CAM', 'Daniele Croce', 325000.0]
['LW', 'Lucas Chiaretti', 775000.0]
['RM', '姜至鹏', 700000.0]
['ST', 'Chris Pontius', 675000.0]
['CM', '仓田 秋', 725000

In [ ]:
#Consulta larga
DisplaySol(GetTeam(better=0.70))

Buscando el mejor equipo de entre 9896 jugadores
1 de 1 equipos encontrados
---  equipo 0 ---
['RM', 'Lionel Andrés Messi Cuccittini', 95500000.0]
['ST', 'Cristiano Ronaldo dos Santos Aveiro', 58500000.0]
['LB', 'Neymar da Silva Santos Junior', 105500000.0]
['CAM', 'Kevin De Bruyne', 90000000.0]
['LM', 'Virgil van Dijk', 78000000.0]
['CM', 'Luka Modrić', 45000000.0]
['RW', "N'Golo Kanté", 66000000.0]
['GK', 'Jordi Alba Ramos', 40000000.0]
['CB', 'Daniel Carvajal Ramos', 38000000.0]
['RB', 'Jadon Sancho', 44500000.0]
['LW', 'Douglas Costa de Souza', 31500000.0]


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=1b69a523-420e-469d-9b69-888f43f9f734' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>